# Bicycles & Weather data engineering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
plt.style.use('ggplot')

%matplotlib inline

In [ ]:
# first parameter is name of the file with data
# next we specify delimiter, it can be either comma, semicolon, sometimes tab
# very often we need to cope with missing data, here we denote it by "NA"
# if the data is already enumerated pandas doesn't need to double the job
bicycles_data = pd.read_csv("../warsaw-bicycles.csv", delimiter=",", na_values="NA", index_col=0)
weather_data = pd.read_csv("../weather.csv", delimiter=",", na_values="NA", index_col=0)

In [ ]:
bicycles_data.head()

In [ ]:
bicycles_data.describe()

In [ ]:
weather_data.head()

In [ ]:
weather_data.describe()

### Data engineering

Now we play around weather dataset in order to extract the day of measurement.

In [ ]:
weather_data["date"] = pd.to_datetime(weather_data["date"], format="%Y-%m-%d")

In [ ]:
weather_data["weekday"] = weather_data["date"].dt.weekday

In [ ]:
weather_data["month"] = weather_data["date"].dt.month

In [ ]:
weather_data.head()

In [ ]:
len(weather_data)

In [ ]:
len(bicycles_data)

We clearly see that there are more measurements of weather states than bicycles counts, so we need limit one dataset to make it consistent.

In [ ]:
bicycles_date_min, bicycles_date_max = bicycles_data["Data"].tolist()[0], bicycles_data["Data"].tolist()[-1]

In [ ]:
weather_data_filtered = weather_data.query("'{}'<=date<='{}'".format(bicycles_date_min, bicycles_date_max))

In [ ]:
weather_data_filtered = weather_data_filtered.reset_index(drop=True)

In [ ]:
weather_data_filtered.index += 1 

Now `weather_data_filtered` should have the same number of rows as `bicycles_data`. You may check its `len` for exercise.

In [ ]:
weather_data_filtered.head()

So we are ready to concatenate two datasets.

In [ ]:
bicycles_weather_data = pd.concat([bicycles_data, weather_data_filtered], axis=1)

Some columns are no longer useful, so we can drop them.

In [ ]:
bicycles_weather_data.drop(['Data', 'state', 'startTyg', 'startM'], axis=1, inplace=True)

In [ ]:
bicycles_weather_data.rename(columns={'value': 'temp'}, inplace=True)

All in all, we end up with dataset which looks like this:

In [ ]:
bicycles_weather_data.head()

In [ ]:
bicycles_weather_data = bicycles_weather_data.set_index("date")

In [ ]:
bicycles_weather_data.index.name=None

In [ ]:
bicycles_weather_data.head()

In [ ]:
bicycles_weather_data = bicycles_weather_data[(bicycles_weather_data.index > '2014-12-15')]

In [ ]:
bicycles_weather_data.to_csv("../bicycles_weather.csv")
#bicycles_weather_data = pd.read_csv("../bicycles_weather.csv", index_col=0)